### IBTrACS data
For historical typhoon events we used IBTrACS data, which comes from the IBTracs project

The IBTrACS project:
    Contains the most complete global set of historical tropical cyclones available
    Combines information from numerous tropical cyclone datasets
    Simplifies inter-agency comparisons by providing storm data from multiple sources in one place
    Combines recent and historical storm data in one dataset


### Parametric wind field modelling 

A parametric representation of the pressure and wind field in tropical cyclones facilitates abstract representation of these meteorological systems using just a few physically meaningful parameters [Depperman, 1947; Holland, 1980; Jelesnianski, 1965; Schloemer, 1954]. These include the central pressure deficit, the maximum wind speed, and the radius to the maximum wind speed.
for our Project we used parametric wind field model based on the Holands 2008 method 

In [ ]:
#%% import libraries from local directory
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from os.path import relpath
from pybufrkit.decoder import Decoder
from geopandas.tools import sjoin
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points

import logging

decoder = Decoder()

from pathlib import Path
# Get the current working directory
current_dir = Path.cwd()

wor_dir = current_dir / Path('../../')


In [ ]:
relative_path = current_dir / Path('../../src')
os.chdir(relative_path)
# Importing local libraries       
from climada.hazard import Centroids, TropCyclone, TCTracks
from climada.hazard.tc_tracks import estimate_roci, estimate_rmw
from climada.hazard.tc_tracks_forecast import TCForecast
from typhoonmodel.utility_fun import track_data_clean, Check_for_active_typhoon, Sendemail, Hwrf_data, plot_intensity, initialize
from typhoonmodel.utility_fun import Rainfall_data
from typhoonmodel.utility_fun.forecast_process import Forecast
from climada.util import coordinates  
from typhoonmodel.utility_fun.settings import *
from typhoonmodel.utility_fun.dynamicDataDb import DatabaseManager

os.chdir(wor_dir)

initialize.setup_logger()

logger = logging.getLogger(__name__)


 

## Define functions

In [3]:
def adjust_tracks(forcast_df):
        track = xr.Dataset(
            data_vars={
                "max_sustained_wind": (
                    "time",
                    0.514444
                    * forcast_df.max_sustained_wind.values,  # conversion from kn to meter/s
                ),
                "environmental_pressure": (
                    "time",
                    forcast_df.environmental_pressure.values,
                ),
                "central_pressure": ("time", forcast_df.central_pressure.values),
                "lat": ("time", forcast_df.lat.values),
                "lon": ("time", forcast_df.lon.values),
                "radius_max_wind": ("time", forcast_df.radius_max_wind.values),
                "radius_oci": ("time", forcast_df.radius_oci.values),
                "time_step": (
                    "time",
                    np.full_like(forcast_df.time_step.values, 3, dtype=float),
                ),
            },
            coords={"time": forcast_df.time.values,},
            attrs={
                "max_sustained_wind_unit": "m/s",
                "central_pressure_unit": "mb",
                "name": forcast_df.name,
                "sid": forcast_df.sid,  # +str(forcast_df.ensemble_number),
                "orig_event_flag": forcast_df.orig_event_flag,
                "data_provider": forcast_df.data_provider,
                "id_no": forcast_df.id_no,
                "basin": forcast_df.basin,
                "category": forcast_df.category,
            },
        )
        track = track.set_coords(["lat", "lon"])
        return track

In [ ]:
#%%######################################################
####### VARIABLES TO SET
#########################################################
year_range = (2006, 2022)

# Laoding PH admin file and setting centroids
# Instead of one grid point for each municipalities: uses a range of gridpoints
# calculates value for each point
# Eventually select a specific point in municipality based on condition
# maximum windspeed and minimum track distance
file_name = "data-raw/gis_data/phl_admin3_simpl2.geojson"

#path = os.path.join(wor_dir, file_name)

admin = gpd.read_file(file_name)

minx, miny, maxx, maxy = admin.total_bounds
print(minx, miny, maxx, maxy)

cent = Centroids()
cent.set_raster_from_pnt_bounds((minx, miny, maxx, maxy), res=0.05)
cent.check()
cent.plot()
plt.show()
plt.close()

In [ ]:
os.listdir('./data/wind_data/')

In [ ]:
# Loads an excel sheet with Local_name, International_Name and year
# Setting directory for typhoon metadata csv file
typhoon_metadata_filename ="data/metadata_typhoons.csv"
typhoon_metadata = pd.read_csv(typhoon_metadata_filename, delimiter=",")
typhoons = list(typhoon_metadata.typhoon.values)
# if there is already wind data in the project folder  wind_data/output
typhoons_with_wind_data = [items.split('_')[0] for items in os.listdir('./data/wind_data/output/')]

typhoons_withoutwind_data = [items for items in typhoons if items not in typhoons_with_wind_data]


In [ ]:
typhoon_events = typhoons_withoutwind_data#list(typhoon_metadata.typhoon.values)
typhoon_events=[event.upper() for event in typhoon_events]


In [ ]:
typoon_event=typhoon_events
sel_ibtracs = TCTracks()
# Set year range for which data should be collected
sel_ibtracs.read_ibtracs_netcdf(
    provider="usa", year_range=year_range, basin="WP", correct_pres=False
)

Typhoons = TCTracks()
# Select typhoons that are in the typhoon event sheet
Typhoons.data = [
    tr for tr in sel_ibtracs.data if (tr.name + tr.sid[:4]) in typoon_event
]

# Plot the typhoon track
ax = Typhoons.plot()
ax.get_legend()._loc = 1
ax.set_title(typoon_event, fontsize=5)
plt.show()
plt.close()

In [ ]:
#%%######################################################
####### START PROCESSING
#########################################################
 
for typhoon_name in typhoon_events:

    typoon_event = [typhoon_name]
    print(typoon_event)

    sel_ibtracs = TCTracks()
    # Set year range for which data should be collected
    sel_ibtracs.read_ibtracs_netcdf(
        provider="usa", year_range=year_range, basin="WP", correct_pres=False
    )

    Typhoons = TCTracks()
    # Select typhoons that are in the typhoon event sheet
    Typhoons.data = [
        tr for tr in sel_ibtracs.data if (tr.name+tr.sid) in typoon_event
    ]

    # Plot the typhoon track
    #ax = Typhoons.plot()
    #ax.get_legend()._loc = 1
    #ax.set_title(typoon_event, fontsize=14)
    #plt.show()
    #plt.close()

    # Select names and storm id's of storms
    #names = [[tr.name, tr.sid] for tr in Typhoons.data]

    df = pd.DataFrame(data=cent.coord)
    df["centroid_id"] = "id" + (df.index).astype(str)
    centroid_idx = df["centroid_id"].values
    ncents = cent.size
    df = df.rename(columns={0: "lat", 1: "lon"})   


    tracks = TCTracks()
    tracks.data = [adjust_tracks(tr) for tr in Typhoons.data]
    tracks.equal_timestep(0.5)

    if tracks.data !=[]:
        # define a new typhoon class
        TYphoon = TropCyclone()
        TYphoon.set_from_tracks(tracks, cent, store_windfields=True)

        # plot intensity
        #TYphoon.plot_intensity(event=Typhoons.data[0].sid)
        #plt.show()
        #plt.close()

        df = pd.DataFrame(data=cent.coord)
        df["centroid_id"] = "id" + (df.index).astype(str)
        centroid_idx = df["centroid_id"].values
        ncents = cent.size
        df = df.rename(columns={0: "lat", 1: "lon"})
        threshold = 0.1

        list_intensity = []
        distan_track = []

        for tr in tracks.data:
            print(tr.name)

            track = TCTracks()
            typhoon = TropCyclone()
            track.data = [tr]
            typhoon.set_from_tracks(track, cent, store_windfields=True)
            windfield = typhoon.windfields
            nsteps = windfield[0].shape[0]
            centroid_id = np.tile(centroid_idx, nsteps)
            intensity_3d = windfield[0].toarray().reshape(nsteps, ncents, 2)
            intensity = np.linalg.norm(intensity_3d, axis=-1).ravel()

            timesteps = np.repeat(tr.time.values, ncents)
            timesteps = timesteps.reshape((nsteps, ncents)).ravel()
            inten_tr = pd.DataFrame(
                {"centroid_id": centroid_id, "value": intensity, "timestamp": timesteps,}
            )

            inten_tr = inten_tr[inten_tr.value > threshold]

            inten_tr["storm_id"] = tr.sid
            list_intensity.append(inten_tr)
            distan_track1 = []
            for index, row in df.iterrows():
                dist = np.min(
                    np.sqrt(
                        np.square(tr.lat.values - row["lat"])
                        + np.square(tr.lon.values - row["lon"])
                    )
                )
                distan_track1.append(dist * 111)
            dist_tr = pd.DataFrame({"centroid_id": centroid_idx, "value": distan_track1})
            dist_tr["storm_id"] = tr.sid
            distan_track.append(dist_tr)

        df_ = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
        df_.crs = {"init": "epsg:4326"}
        df_ = df_.to_crs("EPSG:4326")
        df_admin = sjoin(df_, admin, how="left")
        # To remove points that are in water
        df_admin = df_admin.dropna()

        # For some municipalities, there is missing data
        # For these, fill with the value of the nearest centroid with data observed
        mun_missing = admin[admin["adm3_pcode"].isin(df_admin["adm3_pcode"]) == False]
        print(f"There are {len(mun_missing)} missing municipalities")

        mun_missing_points = gpd.GeoDataFrame(
            mun_missing, geometry=gpd.points_from_xy(mun_missing.glon, mun_missing.glat)
        )
        pts3 = df_admin.geometry.unary_union

        def near(point, pts=pts3):
            # find the nearest point and return the corresponding Place value
            nearest = df_admin.geometry == nearest_points(point, pts)[1]
            # display(df_admin[nearest].centroid_id.values[0])
            return df_admin[nearest].centroid_id.values[0]

        mun_missing_points["centroid_id"] = mun_missing_points.apply(
            lambda row: near(row.geometry), axis=1
        )

        def match_values_lat(x):

            return df_admin["lat"][df_admin["centroid_id"] == x].values[0]

        def match_values_lon(x):

            return df_admin["lon"][df_admin["centroid_id"] == x].values[0]

        def match_values_geo(x):

            return df_admin["geometry"][df_admin["centroid_id"] == x].values[0]

        mun_missing_points["lat"] = mun_missing_points["centroid_id"].apply(
            match_values_lat
        )
        mun_missing_points["lon"] = mun_missing_points["centroid_id"].apply(
            match_values_lon
        )
        mun_missing_points["geometry"] = mun_missing_points["centroid_id"].apply(
            match_values_geo
        )

        df_admin = df_admin[
            [
                "adm3_en",
                "adm3_pcode",
                "adm2_pcode",
                "adm1_pcode",
                "glat",
                "glon",
                "lat",
                "lon",
                "centroid_id",
            ]
        ]

        mun_missing_points = mun_missing_points[
            [
                "adm3_en",
                "adm3_pcode",
                "adm2_pcode",
                "adm1_pcode",
                "glat",
                "glon",
                "lat",
                "lon",
                "centroid_id",
            ]
        ]

        df_admin = pd.concat([df_admin, mun_missing_points])
        mun_missing = admin[admin["adm3_pcode"].isin(df_admin["adm3_pcode"]) == False]
        print(f"There are {len(mun_missing)} missing municipalities")

        df_intensity = pd.concat(list_intensity)
        df_intensity = pd.merge(df_intensity, df_admin, how="outer", on="centroid_id")

        df_intensity = pd.concat(list_intensity)
        df_intensity = pd.merge(df_intensity, df_admin, how="outer", on="centroid_id")

        df_intensity = df_intensity.dropna()

        # Obtains the maximum intensity for each municipality and storm_id combination & also return the count (how often the combination occurs in the set)
        df_intensity = (
            df_intensity[df_intensity["value"].gt(0)]
            .groupby(["adm3_pcode", "storm_id"], as_index=False)
            .agg({"value": ["count", "max"]})
        )
        # rename columns
        df_intensity.columns = [
            x for x in ["adm3_pcode", "storm_id", "value_count", "v_max"]
        ]

        df_track = pd.concat(distan_track)
        df_track = pd.merge(df_track, df_admin, how="outer", on="centroid_id")
        df_track = df_track.dropna()

        # Obtains the minimum track distance for each municipality and storm_id combination
        df_track_ = df_track.groupby(["adm3_pcode", "storm_id"], as_index=False).agg(
            {"value": "min"}
        )
        df_track_.columns = [x for x in ["adm3_pcode", "storm_id", "dis_track_min"]]
        typhhon_df = pd.merge(
            df_intensity, df_track_, how="left", on=["adm3_pcode", "storm_id"]
        )

        # Check if there are duplicates for municipality and storm_id
        duplicate = typhhon_df[
            typhhon_df.duplicated(subset=["adm3_pcode", "storm_id"], keep=False)
        ]
        if len(duplicate) != 0:
            print("There are duplicates, please check")

        fname= typhoon_name.lower()

        file_name = (f"./data/wind_data/output/{fname}_windgrid_output.csv")
        #path = os.path.join(cdir, file_name)
        typhhon_df.to_csv(file_name)

